In [7]:
import json
def load_settings(env):
    with open(f'{env}.json') as f:
              settings = json.load(f)
    return settings

In [8]:
from pprint import pprint

In [9]:
pprint(load_settings('common'))

{'data': {'input_root': '/default/path/inputs',
          'numerics': {'precision': 6, 'type': 'Decimal'},
          'output_root': '/default/path/outputs'},
 'database': {'db_name': 'deepdive', 'port': 5432, 'schema': 'public'},
 'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s '
                    '%(message)s',
          'level': 'info'}}


In [10]:
pprint(load_settings('dev'))

{'data': {'input_root': '/dev/path/inputs',
          'numerics': {'type': 'float'},
          'operators': {'add': '__add__'},
          'output_root': '/dev/path/outputs'},
 'database': {'pwd': 'test', 'user': 'test'},
 'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s '
                    '%(filename)s %(funcName)s %(message)s',
          'level': 'trace'}}


In [12]:
pprint(load_settings('prod'))

{'data': {'input_root': '$DATA_INPUT_PATH', 'output_root': '$DATA_OUTPUT_PATH'},
 'database': {'pwd': '$PG_PWD', 'user': '$PG_USER'}}


In [13]:
from collections import ChainMap

In [14]:
def settings(env):
    common_settings = load_settings('common')
    env_settings = load_settings(env)
    return ChainMap(env_settings,common_settings)

In [15]:
dev= settings('dev')
dev

ChainMap({'data': {'input_root': '/dev/path/inputs', 'output_root': '/dev/path/outputs', 'numerics': {'type': 'float'}, 'operators': {'add': '__add__'}}, 'database': {'user': 'test', 'pwd': 'test'}, 'logs': {'level': 'trace', 'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(filename)s %(funcName)s %(message)s'}}, {'data': {'input_root': '/default/path/inputs', 'output_root': '/default/path/outputs', 'numerics': {'type': 'Decimal', 'precision': 6}}, 'database': {'db_name': 'deepdive', 'schema': 'public', 'port': 5432}, 'logs': {'level': 'info', 'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(message)s'}})

In [16]:
for k,v in dev.items():
    print(k,':' ,v)

logs : {'level': 'trace', 'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(filename)s %(funcName)s %(message)s'}
data : {'input_root': '/dev/path/inputs', 'output_root': '/dev/path/outputs', 'numerics': {'type': 'float'}, 'operators': {'add': '__add__'}}
database : {'user': 'test', 'pwd': 'test'}


In [19]:
def chain_recursive(d1,d2):
    chain = ChainMap(d1,d2)
    for k,v in d1.items():
        if isinstance(v,dict) and k in d2 and isinstance(d2[k],dict):
            chain[k] = chain_recursive(d1[k],d2[k])
    return chain

In [20]:
d1 = load_settings('common')
d2 = load_settings('dev')
dev = chain_recursive(d2,d1)

In [21]:
pprint(dev)

ChainMap({'data': ChainMap({'input_root': '/dev/path/inputs',
                            'numerics': ChainMap({'type': 'float'},
                                                 {'precision': 6,
                                                  'type': 'Decimal'}),
                            'operators': {'add': '__add__'},
                            'output_root': '/dev/path/outputs'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': 'test', 'user': 'test'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'}),
          'logs': ChainMap({'format': '%(asctime)s: %(levelname)s: '
                                      '%(clientip)s %(user)s %(filename)s '
                              

In [25]:
d3 = load_settings('prod')
prod = chain_recursive(d3,d1)

In [26]:
pprint(prod)

ChainMap({'data': ChainMap({'input_root': '$DATA_INPUT_PATH',
                            'output_root': '$DATA_OUTPUT_PATH'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': '$PG_PWD', 'user': '$PG_USER'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'})},
         {'data': {'input_root': '/default/path/inputs',
                   'numerics': {'precision': 6, 'type': 'Decimal'},
                   'output_root': '/default/path/outputs'},
          'database': {'db_name': 'deepdive', 'port': 5432, 'schema': 'public'},
          'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s '
                             '%(user)s %(message)s',
                   'level': 'info'}})


In [29]:
prod['logs']['level']

'info'

In [30]:
dev['logs']['level']

'trace'

In [31]:
def settings(env):
    common_settings = load_settings('common')
    env_settings = load_settings(env)
    return chain_recursive(env_settings,common_settings)

In [34]:
prod = settings('prod')
pprint(prod)

ChainMap({'data': ChainMap({'input_root': '$DATA_INPUT_PATH',
                            'output_root': '$DATA_OUTPUT_PATH'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': '$PG_PWD', 'user': '$PG_USER'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'})},
         {'data': {'input_root': '/default/path/inputs',
                   'numerics': {'precision': 6, 'type': 'Decimal'},
                   'output_root': '/default/path/outputs'},
          'database': {'db_name': 'deepdive', 'port': 5432, 'schema': 'public'},
          'logs': {'format': '%(asctime)s: %(levelname)s: %(clientip)s '
                             '%(user)s %(message)s',
                   'level': 'info'}})
